#Assignment 08: Advanced Sequence Applications

This Week's assignment is to train sequence models on the `Cafe Chatbot Data`.

Before starting copy this file and work on your own copy by following the below steps: <br>
`File > Save Copy in Drive`. Then add your name to the file (e.g., Assignment 07: Sequence Modeling - Zahraa Dhafer).

**DATASET**
This dataset consists of three csv files, you'll be working on the conversationo CSV file which contains two columns of questions and answers.<br><br>
**Submission Deadline: Saturday, 3/12/2022 at 3:00 PM**

**Requirements:**

1. Import all necessary libraries for the sequence modeling project.
2. Download the dataset (the link is provided below).
3. Read data from CSV file.
4. Prepare the data:
*   Clean the sentences by removing special characters.
*   Add a start and end token to each sentence.
*   Tokenize the sentences.
*   Pad each sentence to a maximum length.
<br>
5. Create the Data Pipeline for the Model
6. Create the Seq2Seq Model Architecture.<br>
**Note**: Use Adam optimizer and the appropriate loss function.
7. Create the Optimizer and the Loss Function.
8. Create the Training Step.
9. Create the Training Loop

**Note:** you can overfit your model.<br>
**HINTS:**
Set the new hyperparameters like vocabulary size, input length (i.e. max sequence length) in a separate cell after the import cell in your notebook (failing to do so will affect your style score)

**Note:** To get the best performance from the model, manually tune the hyperparameters of the model. 

Find relevant links below:<br>

[Assignment Colab File](https://colab.research.google.com/drive/1N6IcInQFF0iJdl8pruUvb4-rAacE0IBC?usp=sharing)<br>

[Dataset](https://www.kaggle.com/sonalibhoir/cafe-chatbot-dataset?select=conversationo.csv)

[Submission Form](https://docs.google.com/forms/d/e/1FAIpQLSf-WkcxjmZcxdUcu5OG7ZxFOsQu-qBy-u1UFNPRkWmjeW7w3g/viewform?usp=pp_url)<br>


Good luck and feel free to ask any questions in the or on the Questions channel.

In [ ]:
!rm /content/conversationo.csv
!rm /content/food.csv
!rm /content/Item_to_id.csv

## 1- Import all necessary libraries for the sequence modeling project

In [ ]:
# write your code below
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import gradio as gr
from sklearn import model_selection


In [ ]:
buffer_size=937
vocab_size=64

units=300
batch_size=32
embedding_dim = 256

# units = 1024

## 2- Download the dataset

In [ ]:
# write your code below

!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: Item_to_id.csv          
  inflating: conversationo.csv       
  inflating: food.csv                


## 3- Read data from CSV file

In [ ]:
from pandas.core.frame import DataFrame
# write your code below
pf=pd.read_csv('/content/conversationo.csv')
df=DataFrame(pf)
df

,Question,answer
0,hey,Hello! How may I help you.
1,do u have coffee,"Yes sir Simple Coffee ,Cappuchino, Americano,..."
2,i will take one espresso and 5 americano,Sir thanks for your order. You have ordered 1 ...
3,anything special,"We have coffe,pastries,puff pastries and milks..."
4,suggest something,"We have coffe,pastries,puff pastries and milks..."
...,...,...
974,what is price of French Coffee,"Its our one of best, you can enjoy it at just ..."
975,what is price of Iced Coffee Late,"Its our one of best, you can enjoy it at just ..."
976,what is price of Latte Macchiato,"Its our one of best, you can enjoy it at just ..."
977,what is price of Wainans Choco Coffee,"Its our one of best, you can enjoy it at just ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979 entries, 0 to 978
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  961 non-null    object
 1   answer    939 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB


In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 937 entries, 0 to 978
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  937 non-null    object
 1   answer    937 non-null    object
dtypes: object(2)
memory usage: 22.0+ KB


In [ ]:
df.Question

0                                           hey
1                              do u have coffee
2      i will take one espresso and 5 americano
3                              anything special
4                             suggest something
                         ...                   
974             what is price of French Coffee 
975           what is price of Iced Coffee Late
976            what is price of Latte Macchiato
977       what is price of Wainans Choco Coffee
978                             book me a table
Name: Question, Length: 937, dtype: object

##4- Prepare the data:


### Clean the sentences by removing special characters

In [ ]:
# write your code below
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^\w]',' ',text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(filter(lambda x:x[0]!="@", text.split()))
    return text
df.Question=df.Question.map(clean_text)
df.answer =df.answer.map(clean_text)


### Add a start and end token to each sentence

In [ ]:
# write your code below
def startend(text):
  text=f'<start> {text} <end>'
  return text

df.Question=df.Question.map(startend)
df.answer =df.answer.map(startend)
df

,Question,answer
0,<start> hey <end>,<start> hello how may i help you <end>
1,<start> do u have coffee <end>,<start> yes sir simple coffee cappuchino ameri...
2,<start> i will take one espresso and americano...,<start> sir thanks for your order you have ord...
3,<start> anything special <end>,<start> we have coffe pastries puff pastries a...
4,<start> suggest something <end>,<start> we have coffe pastries puff pastries a...
...,...,...
974,<start> what is price of french coffee <end>,<start> its our one of best you can enjoy it a...
975,<start> what is price of iced coffee late <end>,<start> its our one of best you can enjoy it a...
976,<start> what is price of latte macchiato <end>,<start> its our one of best you can enjoy it a...
977,<start> what is price of wainans choco coffee ...,<start> its our one of best you can enjoy it a...


### Tokenize the sentences

In [ ]:
# write your code below
def token_texts(texts):
  token_zer=tf.keras.preprocessing.text.Tokenizer(oov_token='OOV',
                                              filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
  token_zer.fit_on_texts(texts)
  tensor=token_zer.texts_to_sequences(texts)

  return tensor,token_zer

tensor_Q,token_Q=token_texts(df.Question)
tensor_a,token_a = token_texts(df.answer)


### Pad each sentence to a maximum length

In [ ]:
# write your code below
def padding_text(texts):
  
  pad=tf.keras.preprocessing.sequence.pad_sequences(sequences=texts,padding='post')
  return pad
padd_Q=padding_text(tensor_Q)
padd_a=padding_text(tensor_a)


In [ ]:
padd_a.shape

(937, 54)

In [ ]:

max_sequence_Q=padd_Q.shape[1]
max_sequence_a=padd_a.shape[1]


In [ ]:
token_Q.word_index['hey']
tensor_a[0]

[2, 47, 48, 23, 22, 49, 4, 3]

## 5- Create the Data Pipeline for the Model

In [ ]:
# x_train,x_test,y_train,y_test=model_selection.train_test_split(padd_Q,padd_a,random_state=42,
#                                                 test_size=0.1)

In [ ]:
# x_train.shape

In [ ]:
# write your code below
def create_data(x,y):
  data=tf.data.Dataset.from_tensor_slices((x,y))
  data=data.shuffle(buffer_size=buffer_size)
  data=data.batch(batch_size=batch_size,drop_remainder=True)
  data=data.prefetch(tf.data.AUTOTUNE)

  return data
train_dataset=create_data(padd_Q,padd_a)
# test_dataset=create_data(x_test,y_test)


In [ ]:
# get the first batch of the training data 
for question, answer in train_dataset.take(1):
  print(f"question:{question.shape}\n{question}")

  print(f"answer:{answer.shape}\n{answer}")
  question_sample = question
  answer_sample = answer

question:(32, 49)
[[ 2  4  6 ...  0  0  0]
 [ 2  4  7 ...  0  0  0]
 [ 2 35  3 ...  0  0  0]
 ...
 [ 2  4 14 ...  0  0  0]
 [ 2  4  7 ...  0  0  0]
 [ 2 16 25 ...  0  0  0]]
answer:(32, 54)
[[ 2 20  6 ...  0  0  0]
 [ 2 10  7 ...  0  0  0]
 [ 2 47 48 ...  0  0  0]
 ...
 [ 2 20  6 ...  0  0  0]
 [ 2 10  7 ...  0  0  0]
 [ 2 24 50 ...  0  0  0]]


In [ ]:
# create nesseary variables
# the size of the vocabulary for the question and the answer
vocab_inp_size= len(token_Q.word_index)+1
vocab_tar_size=  len(token_a.word_index)+1
print(vocab_inp_size)
print(vocab_tar_size )



166
201


##6- Create the Seq2Seq Model Architecture

###Encoder Model

In [ ]:
# write your code below
class Encoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,encoder_units,batch_size):
      super(Encoder,self).__init__()

      self.encoder_units=encoder_units

      self.embedding=tf.keras.layers.Embedding(vocab_size,embedding_dim,
                                               mask_zero=True)
      self.batch_size=batch_size

      self.gru=tf.keras.layers.GRU(units=self.encoder_units,return_sequences=True,
                                   return_state=True,recurrent_initializer='glorot_uniform')


  def call(self,x,hidden):
    x=self.embedding(x)
    out,state=self.gru(x,initial_state=hidden)

    return out,state


  def initializer_hiddenstate(self):
    return tf.zeros(shape=(self.batch_size,self.encoder_units))      


### Decoder Model

In [ ]:
# write your code below
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,decoder_units,batch_size):
      super(Decoder,self).__init__()

      self.decoder_units=decoder_units

      self.batch_size=batch_size
      

      self.embedding=tf.keras.layers.Embedding(vocab_size,output_dim=embedding_dim,mask_zero=True)

      self.gru=tf.keras.layers.GRU(units=self.decoder_units,recurrent_initializer='glorot_uniform',
                                   return_sequences=True,
                                   return_state=True)
      
      self.fc=tf.keras.layers.Dense(vocab_size)



  def call(self,x,hidden):
    x=self.embedding(x)
    out,state=self.gru(x,hidden)
    out=tf.reshape(out,(-1,out.shape[2]))

    x=tf.nn.softmax(self.fc(out))

    return x,hidden


In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)

# sample input
# inilize the hidden state with zeros 
# the shape of hidden is (batch_size, units)
sample_hidden = encoder.initializer_hiddenstate()
sample_output, sample_hidden = encoder(question, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (32, 49, 300)
Encoder Hidden state shape: (batch size, units) (32, 300)


In [ ]:
decoder=Decoder(vocab_tar_size,embedding_dim,units,batch_size)

sample_decoder_out, _ = decoder(tf.random.uniform(shape=(batch_size,1))
                                              ,sample_hidden)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_out.shape))

Decoder output shape: (batch_size, vocab size) (32, 201)


## 7- Create the Optimizer and the Loss Function

In [ ]:
# write your code below

optimizer = tf.keras.optimizers.Adam()
# create the loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction='none')

# define the loss function for the training
def loss_function(real, pred):
  # create the mask to ignore the padding tokens
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  # mask shape == (batch_size, sequence_length)
  # calculate the loss
  loss_ = loss_object(real, pred)
  # mask the loss
  # how the mask works:
  # if the value is 1, the loss is calculated
  # if the value is 0, the loss is ignored
    #[1,1,1,1,1,1,0,0,0,0,0] mask
    # *
    #[2,6,2,1,6,3,2,1,5,7,9] input
    # =
    #[2,6,2,1,6,3,0,0,0,0,0] output
  mask = tf.cast(mask, dtype=loss_.dtype)
  # mask shape == (batch_size, sequence_length)
  
  loss_ *= mask
  # calculate the average loss per batch 
  return tf.reduce_mean(loss_)


In [ ]:
# create the training metric 
train_loss = tf.metrics.Mean(name='train loss')
# create the testing metric 
test_loss =tf.metrics.Mean(name='test loss')

##8- Create the Training Step

## Computing Gradients Using Autodiff



In [ ]:

@tf.function
def train_setp(inputs,target,enc_hidden):
  loss=0
  
  with tf.GradientTape() as tape:

    enc_output , enc_hidden = encoder(inputs,enc_hidden)

    dec_hidden= enc_hidden

    dec_input = tf.expand_dims([token_a.word_index['<start>']] * inputs.shape[0],1)

    for t in range(1,target.shape[1]):
      predictions , dec_hidden = decoder(dec_input,dec_hidden)
      loss+=loss_function(target[:,t],predictions)
      dec_input=tf.expand_dims(target[:,t],1)

    batch_loss = (loss/int(target.shape[1]))
    variables=encoder.trainable_variables + decoder.trainable_variables

    gradients=tape.gradient(loss,variables)

    optimizer.apply_gradients(zip(gradients,variables))

    train_loss(batch_loss)
    return batch_loss



## 9- Create the Training Loop

In [ ]:

# write your code below
epochs = 200

old_test_loss = 1000000

for epoch in range(epochs):
  train_loss.reset_states()
  test_loss.reset_states()

  enc_hidden = encoder.initializer_hiddenstate()
  
  steps_per_epoch = padd_a.shape[0]//batch_size
  target = steps_per_epoch
  bar = tf.keras.utils.Progbar(target)

  count = 0
  for(batch,(input,target)) in enumerate(train_dataset):
    count += 1
    batch_loss = train_setp(input,target,enc_hidden)
    bar.update(count)



  if old_test_loss> test_loss.result():
        # set the old test loss to the test loss 
        old_test_loss= test_loss.result()
        encoder.save(filepath='/content/models/encoder')
        decoder.save(filepath='/content/models/decoder')
        print('Model is saved')
    # print the training and testing loss
  print('>' *50)
  print(f'Epoch #{epoch + 1}')
  print(f'Training Loss {train_loss.result()}')
  print('<' * 50)


29/29 [==============================] - 163s 213ms/step


INFO:tensorflow:Assets written to: /content/models/encoder/assets


INFO:tensorflow:Assets written to: /content/models/encoder/assets


INFO:tensorflow:Assets written to: /content/models/decoder/assets


INFO:tensorflow:Assets written to: /content/models/decoder/assets


Model is saved
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Epoch #1
Training Loss 1.0472263097763062
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
29/29 [==============================] - 6s 216ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Epoch #2
Training Loss 0.696025013923645
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
29/29 [==============================] - 6s 216ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Epoch #3
Training Loss 0.42190974950790405
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
29/29 [==============================] - 6s 217ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Epoch #4
Training Loss 0.26402512192726135
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
29/29 [==============================] - 6s 216ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Epoch #5
Training Loss 0.1761874258518219
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
29/29 [==============================] - 6s 

In [ ]:
def chatbot(question):
  question=clean_text(question)
  question=startend(question)

  inputs = token_Q.texts_to_sequences([question])
  inputs=tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       maxlen=max_sequence_Q,
                                                       padding='post')
  
  hidden = [tf.zeros((1,units))]
  out_hidden,  enc_hidden = encoder(inputs,hidden)

  dec_hidden = enc_hidden

  dec_input=tf.expand_dims([token_a.word_index['<start>']],0)

  result=''

  for t in range(max_sequence_a):
    predictions,dec_hidden = decoder(dec_input,dec_hidden)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += token_a.index_word[predicted_id] + ' '

    if token_a.index_word[predicted_id] == '<end>':
      result = result.replace('<start> ', '')
      result = result.replace(' <end> ','')
      # remove the <start> and <end> tokens from the sentence string
      question = question.replace('<start> ', '')
      question = question.replace(' <end>', '')
      return  question, result

    # using the predicted word as the next decoder input
    dec_input = tf.expand_dims([predicted_id], 0)
  # remove the <start> and <end> tokens from the result string
  result = result.replace('<start> ', '')
  result = result.replace('<end>','')
  # remove the <start> and <end> tokens from the sentence string
  question = question.replace('<start> ', '')
  question = question.replace('<end>', '')
  

  return question,result










In [ ]:

chatbot("can I have a cup of coffee ?")

('can i have a cup of coffee', 'sir thanks for sometime')

In [ ]:
chatbot('i will take one espresso and americano')

('i will take one espresso and americano', 'sir thanks for sometime')

In [ ]:
chatbot('anything special')

('anything special', 'we have')

In [ ]:
df.answer[2]

In [ ]:
!pip install gradio

In [ ]:
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.inputs.Textbox(),
    outputs=gr.outputs.Textbox(),
    title='ChatBot',
    description='talk with me here pls',
    theme='dark',
)
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://36185.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7fc205a25ed0>,
 'http://127.0.0.1:7860/',
 'https://36185.gradio.app')

In [ ]:
def chat(message, history):
    obj=chatbot(message)
    history = history or []
  
    
    history.append((obj))
    return history, history


iface = gr.Interface(
    chat,
    ["text", "state"],
    ["chatbot", "state"],
    allow_screenshot=False,
    allow_flagging="never",
    
)
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://37775.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7fc205a25ed0>,
 'http://127.0.0.1:7862/',
 'https://37775.gradio.app')